In [86]:
import torch
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
from langchain.embeddings import OpenAIEmbeddings

In [9]:
load_dotenv('./.env')
os.environ.get('PINECONE_ENV')

'gcp-starter'

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

if num_gpus > 0:
    print(f"GPUs available: {num_gpus}")
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
else:
    print("There is no GPU available. Using CPU")

GPUs available: 1
GPU 0: NVIDIA RTX A4000


In [4]:
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


In [5]:
output = llm('explique como funções python funcionam')

In [6]:
print(output)



Em Python, uma função é um bloco de código que pode ser reutilizado para executar uma tarefa específica. Uma função começa com a palavra-chave def seguida de um nome de função e parênteses. Dentro dos parênteses, você pode fornecer argumentos ou parâmetros que a função usa quando é chamada.

Em seguida, a função contém um bloco de código que é executado quando a função é chamada. Esse bloco de código pode incluir qualquer coisa, desde instruções simples até instruções mais complexas. Por fim, a função retorna um resultado, que pode ser qualquer tipo de valor, desde um número até um objeto.

Essencialmente, uma função é uma maneira de encapsular um conjunto de instruções para que elas possam ser reutilizadas várias vezes. Isso significa que você pode escrever um código uma vez e chamá-lo várias vezes sem ter que digitar o código novamente.


In [7]:
output = llm.generate([
    '...é a capital do Brasil',
    'qual é a formula da área de um círculo?'
])

In [8]:
print(output)

generations=[[Generation(text='?\n\nBrasília é a capital do Brasil.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nA área de um círculo é calculada usando a seguinte formula: A = π * r², onde A é a área e r é o raio do círculo.', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'total_tokens': 91, 'prompt_tokens': 23, 'completion_tokens': 68}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('0fb39415-29c5-48ff-8591-b5e94dc3330c')), RunInfo(run_id=UUID('1c9e54ea-cc01-4776-9e64-f64d3e167adc'))]


In [11]:
print(output.generations[1][0].text)



A área de um círculo é calculada usando a seguinte formula: A = π * r², onde A é a área e r é o raio do círculo.


In [14]:
output = llm.generate(['escreva um slogan original para um restaurante acreano que venda Rabada no Tucupí'] * 3)

In [15]:
for o in output.generations:
    print(o[0].text, end='')



"Saboreie o sabor do Acre: Rabada no Tucupí!"

"Sabor Acreano: Rabada no Tucupí, Delícia em Cada Gole!"

"Rabada no Tucupí, sabor acreano autêntico!"

### ChatModels: GPT-3.5-Turbo e GPT-4

In [26]:
chat = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.5,
    max_tokens=1024
)

In [27]:
messages = [
    SystemMessage(content="Voce e um especialista em machine learning que responde tudo em portugues do Brasil"),
    HumanMessage(content="explique em um paragrafo o que e machine learning.")
]

In [28]:
output = chat(messages)

In [29]:
print(output.content)

Machine learning, ou aprendizado de máquina, é uma área da inteligência artificial que se baseia em algoritmos e modelos estatísticos para permitir que os computadores aprendam e melhorem a partir de dados, sem serem explicitamente programados. É um processo de treinamento em que os sistemas são expostos a grandes volumes de dados, identificam padrões e tendências, e utilizam essas informações para tomar decisões ou realizar previsões de forma autônoma. O objetivo do machine learning é capacitar as máquinas a aprenderem e se adaptarem, proporcionando soluções mais eficientes e precisas em diversas áreas, como reconhecimento de padrões, análise de dados, processamento de linguagem natural, entre outras.


### Prompt Templates

In [35]:
meu_template = """Você é um virologista experiente.
Escreva 5 frases sobre o vírus {virus} no idioma {idioma}."""

prompt = PromptTemplate(
    input_variables=['virus', 'idioma'],
    template=meu_template
)

print(prompt)

input_variables=['idioma', 'virus'] template='Você é um virologista experiente.\nEscreva 5 frases sobre o vírus {virus} no idioma {idioma}.'


In [38]:
llm = OpenAI(
    model_name='text-davinci-003',
    temperature=0.7
)

output = llm(prompt.format(virus='covid-19', idioma='portugues do brasil'))

In [39]:
print(output)



1. O vírus covid-19 pode ser transmitido de pessoa para pessoa através de contato próximo ou por gotículas de saliva. 
2. É importante praticar medidas de higiene frequentes, como lavar as mãos ou usar álcool em gel, para evitar a propagação do vírus.
3. O vírus covid-19 pode causar sintomas leves a graves, como tosse, febre, falta de ar e dificuldades respiratórias.
4. As pessoas de maior risco para a doença são aquelas com idade avançada ou que possuem doenças preexistentes.
5. A melhor maneira de prevenir a propagação do vírus covid-19 é evitando contato próximo com outras pessoas e seguindo as orientações das autoridades de saúde.


### Simple Chains

In [41]:
llm = ChatOpenAI(
    model_name='gpt-4',
    temperature=0.5
)

meu_template = """Você é um virologista experiente.
Escreva um resumo sobre o vírus {virus} no idioma {idioma}."""

prompt = PromptTemplate(
    input_variables=['virus', 'idioma'],
    template=meu_template
)

chain = LLMChain(llm=llm, prompt=prompt)

In [42]:
output = chain.run({
    'virus':'covid-19','idioma':'portugues do brasil'
})

In [43]:
print(output)

O COVID-19 é uma doença infecciosa causada pelo coronavírus SARS-CoV-2. Foi identificado pela primeira vez em Wuhan, China, em dezembro de 2019 e, desde então, desencadeou uma pandemia global. O vírus se espalha principalmente através de gotículas produzidas quando uma pessoa infectada tosse, espirra ou fala. Também pode se espalhar tocando uma superfície ou objeto contaminado e depois tocando a boca, nariz ou olhos.

Os sintomas do COVID-19 variam em gravidade, desde infecções assintomáticas ou leves a casos graves ou críticos. Os sintomas mais comuns incluem febre, tosse seca e cansaço, mas também podem incluir dores no corpo, dor de garganta, perda de paladar ou olfato, e dificuldade para respirar. Em casos graves, pode levar a pneumonia, síndrome respiratória aguda grave e morte.

A prevenção da COVID-19 envolve medidas de saúde pública para reduzir a transmissão do vírus. Isso inclui distanciamento físico, uso de máscaras faciais, higienização frequente das mãos, cobrir a boca ao 

### Sequencial Chains

In [18]:
#--- primeira chain ---

llm1 = OpenAI(
    model_name='text-davinci-003',
    temperature=0.7,
    max_tokens=1024
)

prompt1 = PromptTemplate(
    input_variables=['conceito'],
    template="""
    Voce e um cientista experiente e programador Python.
    Escreva uma funcao que implementa o conceito de {conceito}"""
)

chain1 = LLMChain(llm=llm1, prompt=prompt1)

In [26]:
# --- segunda chain ---

llm2 = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=1.2
)

prompt2 = PromptTemplate(
    input_variables=['function'],
    template="""Dada a funcao Python {function}, descreva como funciona da forma mais detalhada possivel e por ultimo exiba a o codigo da funcao"""
)

chain2 = LLMChain(llm=llm2, prompt=prompt2)

In [27]:
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('regressao linear')



> Entering new SimpleSequentialChain chain...
.

def linear_regression(x, y):
    '''Esta funcao implementa o conceito de regressao linear.
    
    Ela calcula a linha de regressao que melhor se ajusta aos dados x e y,
    usando o metodo dos minimos quadrados.
    
    Args:
        x: Lista de valores de x.
        y: Lista de valores de y.
        
    Retorna:
        A inclinacao (m) e o intercepto (b) da linha de regressao.
    '''
    N = len(x)
    
    # Calcula a media dos valores de x e y
    x_mean = sum(x)/N
    y_mean = sum(y)/N
    
    # Calcula os coeficientes da regressao linear
    numerador = 0
    denominador = 0
    for i in range(N):
        numerador += (x[i] - x_mean) * (y[i] - y_mean)
        denominador += (x[i] - x_mean)**2
    m = numerador/denominador
    b = y_mean - m*x_mean
    
    return m, b
A função `linear_regression` implementa o conceito de regressão linear, que é um método estatístico para encontrar a melhor linha de ajuste aos dados.
A funçã

In [28]:
print(output)

A função `linear_regression` implementa o conceito de regressão linear, que é um método estatístico para encontrar a melhor linha de ajuste aos dados.
A função recebe duas listas de valores, `x` e `y`, que representam os pontos no gráfico. Ela calcula a inclinação (m) e o intercepto (b) da linha de regressão que melhor se ajusta aos dados usando o método dos mínimos quadrados.

A função começa obtendo o número de elementos nas listas `x` e `y`. Isso é armazenado em `N` por meio da função `len()`.

Em seguida, a função calcula a média dos valores de `x` e `y`. Isso é feito somando todos os elementos da lista e dividindo pelo total de elementos. A média de `x` é armazenada em `x_mean`, e a média de `y` é armazenada em `y_mean`.

Agora a função inicia o cálculo dos coeficientes da regressão linear. Ela usa um loop `for` para percorrer todos os elementos das listas `x` e `y`. Para cada elemento, ela calcula as diferenças em relação às médias de `x` e `y` e as multiplica. Esses valores são 

### LangChain Agents

In [7]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonAstREPLTool(),
    verbose=True
)

In [5]:
agent_executor.run('calcule a raiz quadrada do fatorial de 20 e exiba com quatro casas decimais')



> Entering new AgentExecutor chain...
 I need to calculate the square root of the factorial of 20
Action: python_repl_ast
Action Input: from math import factorial; print(f"{round(factorial(20)**0.5, 4)}")
Observation: 1559776268.6285

Thought: I now know the final answer
Final Answer: 1559776268.6285

> Finished chain.


'1559776268.6285'

### Splitting e Embedding de texto

In [84]:
with open('docs/CLT.txt') as f:
    clt = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

In [79]:
chunks = text_splitter.create_documents([clt])

In [96]:
len(chunks)

17153

In [98]:
def embedding_cost(texts):
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    total_em_dolar = total_tokens / 1000 * 0.0001
    print(f'Total de tokens: {total_tokens}')
    print(f'Custo de Embedding em USD: {total_em_dolar:.6f}')
    print(f'Custo de Embedding em R$: {total_em_dolar * 4.94:.6f}')

In [99]:
embedding_cost(chunks)

Total de tokens: 281199
Custo de Embedding em USD: 0.028120
Custo de Embedding em R$: 0.138912


In [88]:
embeddings = OpenAIEmbeddings()

In [93]:
vector = embeddings.embed_query(chunks[0].page_content)

In [94]:
print(vector)

[-0.01184240307476679, -0.008390643018023862, 0.012742862584434661, -0.03075204718985644, -0.01174689966643984, 0.015348736773788719, -0.016972290990056402, 0.0071354571618490025, -0.02107893196017952, -0.009679936236469835, 0.003990671747449589, 0.008656687428658124, 0.004635318356672575, 0.0018009184372591125, 0.001246658549945776, 0.014625640204543688, 0.01138535184747863, -0.0065385617911281856, -0.017777248022719936, -0.007735763501627584, -0.001606501117208757, 0.0006740653442312532, 0.00607127812232804, -0.005300430314580841, 0.011092020615704868, -0.0203558363222571, 0.0009627068427965694, -0.023220932984129892, 0.02996073403342396, 0.011590001609040973, 0.0159490431135673, 0.00444090103662222, -0.02406681978284137, -0.028378109583204227, -0.0262224646830228, -0.030561040373202543, -0.008479324488235284, 0.0070126673980438585, 0.015594316301399009, -0.00391222225308885, 0.021924816896245782, -0.035636355577535764, -0.0035609068756396263, -0.01908700612418988, -0.021010714441669